In [1]:
%load_ext autoreload
%autoreload 2
from Methods import JobOffers,dataframeHelpers

In [2]:
import os
import nltk
import re
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

In [3]:
autoreload JobOffers,dataframeHelpers

In [4]:
# Only run this if you want to redo the translations
# JobOffers.writeTranslatedJobs(-1, firstEntry=0, writeEvery=1000)

In [5]:
allJobsDf = dataframeHelpers.buildJobsDF(["Data/output_V1.1_%i.translate.json" % i for i in range(0, 9000, 1000)])

In [6]:
allJobsDf["NumSentences"] = allJobsDf.apply(lambda x: len(x["JOB_CONTENT"]), axis=1)
allJobsDf["NumCharacters"] = allJobsDf.apply(lambda x: sum([len(y) for y in x["JOB_CONTENT"]]), axis=1)
allJobsDf["CANTON1"] = allJobsDf.apply(lambda x: x["CANTON"][0], axis=1)

In [7]:
allJobsDf["COMPANY_SIZE"] = allJobsDf["COMPANY_SIZE"].replace("Grossunternehmen", "Large")
allJobsDf["COMPANY_SIZE"] = allJobsDf["COMPANY_SIZE"].replace("Kleinstunternehmen", "Micro")
allJobsDf["COMPANY_SIZE"] = allJobsDf["COMPANY_SIZE"].replace("Kleinunternehmen", "Small")
allJobsDf["COMPANY_SIZE"] = allJobsDf["COMPANY_SIZE"].replace("Mittelunternehmen", "Midsize")

In [8]:
allJobsDf

,JOB_ID,ISCO,JOB_CONTENT,ORIGINAL_CONTENT,TRANSLATED_JOBS,CANTON,COMPANY_SIZE,NumSentences,NumCharacters,CANTON1
0,12220025,21420,[We are a successful and independent engineeri...,<p>Wir sind ein erfolgreiches und unabhängiges...,We are a successful and independent engineerin...,[BE],Micro,11,1077,BE
1,10814567,21420,[We are a prestigious and successful engineeri...,Wir sind eine renommierte und erfolgreiche Ing...,We are a prestigious and successful engineerin...,[BE],Micro,18,1694,BE
2,12401674,21420,[Bill Weyermann Partner AG has been producing ...,Bill Weyermann Partner AG erbringt seit über\n...,Bill Weyermann Partner AG has been producing e...,[BE],Small,10,788,BE
3,12473085,62100,[Administration Communale de Valderuzforstarti...,Administration communale de Val-de-RuzForstwar...,Administration Communale de Valderuzforstartin...,[NE],Micro,6,144,NE
4,12905109,26220;24210,[The Straumann Group is experiencing internati...,Le groupe Straumann connaît un succès internat...,The Straumann Group is experiencing internatio...,[BE],Midsize,24,2377,BE
...,...,...,...,...,...,...,...,...,...,...
8560,21415281,21490;72001,[Engineer Development Mechanics wm APGASMurten...,<h1>Engineer Development Mechanics (w/m)</h1>\...,Engineer Development Mechanics wm APGASMurten ...,[FR],Large,14,2505,FR
8561,19996189,21490,[For our teams specialized on Pharma Laborator...,"<p>Für unsere auf Pharma, Labor, Spital und an...",For our teams specialized on Pharma Laboratory...,[BE],Small,16,2157,BE
8562,20867808,25110;12190,[Audemars Piguet is the oldest watchmaker alwa...,<p>Audemars Piguet est la plus ancienne manufa...,Audemars Piguet is the oldest watchmaker alway...,[VD],Midsize,21,2900,VD
8563,21486595,21310;23100;21200,[The Institute of Social and Preventive Medici...,<p>The Institute of Social and Preventive Medi...,The Institute of Social and Preventive Medicin...,[BE],Large,30,3160,BE


In [9]:
#Vika's file
VikaDf = pd.read_csv("Data/Environ_NonEnviron_Roberta_MJ.csv", sep=";")

In [10]:
#Emily's file
EmilyDf = pd.read_csv("Data/EmilyMJDifferentClassifications.csv", sep=";")

In [11]:
#All green's
GreenDf = pd.read_csv("Data/allGreenTranslated.csv", sep=";")

In [12]:
GreenDf = GreenDf.rename(columns ={"KANTONE":"CANTON"}) 
GreenDf = GreenDf.rename(columns ={"FIRMENGROESSE":"COMPANY_SIZE"}) 
GreenDf = GreenDf.rename(columns ={"ID":"JOB_ID"}) 

In [13]:
GreenDf

,Unnamed: 0,JOB_ID,ISCO,SKILLS,BRANCHE,ARBEITSORT,CANTON,COMPANY_SIZE,BERUFSERFAHRUNG,AUSBILDUNGSNIVEAU,ZUDATUM,ABDATUM,FIRMA,JOBTITEL,JOBS,CONTENT,CLEANED_JOBS,TRANSLATED_JOBS,MJ
0,0,20533641,20000,Kommunikationsfähigkeit;Umwelt;Python;Umsicht;...,Öffentliche Verwaltung,CH-1211 Genève,GE,Kleinstunternehmen,NaN,MASTER,43971,44001.0,Office fédéral de météorologie et de climatolo...,Collaboratrice/ Collaborateur scientifique 80%,Wissenschaftlicher Assistent,<p>L'Office fédéral de météorologie et de clim...,L'Office fédéral de météorologie et de climato...,The Federal Meteorology and Climatology Office...,NaN
1,1,20141197,21430,Kommunikationsfähigkeit;Nachhaltigkeit;Modelli...,Umwelttechnik;Rechts- und Unternehmensberatung,Lausanne,VD,Mittelunternehmen,NaN,BACHELOR;MASTER;BASICEDUCATION;MATURA;CERTIFIC...,43892,44031.0,Quantis International SA,Senior Sustainability Consultant,Umweltberater,<h1>Senior Sustainability Consultant</h1> <h2>...,Senior Sustainability Consultant Why youll lov...,Senior Sustainability Consultant Why youll lov...,NaN
2,2,20912180,21430,Hydraulik;Eigenverantwortung;Verantwortungsbew...,Personalberatung,1920 Martigny,VS,Grossunternehmen,NaN,BACHELOR;MASTER;DIPLOMA;UAS;UNIVERSITY,44047,44054.0,Adecco,Ingénieur en environnement EPF ou HES orientat...,Umweltschutztechniker,À la recherche d'une nouvelle opportunité en t...,À la recherche d'une nouvelle opportunité en t...,In search of a new opportunity as an environne...,NaN
3,3,19968711,21310;21410,Umwelt;Ingenieurwesen;Mitarbeitermotivation;Un...,Rechts- und Unternehmensberatung,CH-1203 Genève,GE,Kleinunternehmen,Ingenieur-Jobs;Techniker-Jobs;Postdienste von ...,BACHELOR;MASTER;COLLEGE;DIPLOMA,43864,44004.0,Ecotec Environnement SA,Un·e Spécialiste en renaturation de cours d'ea...,Biologe;Wirtschaftsingenieur,<h1>Offre d'emploi Un·e Spécialiste en renatur...,Offre d'emploi Une Spécialiste en renaturation...,Job Posting A Specialist in Renaturation of Co...,NaN
4,4,20506366,23000,F#;Garten- und Landschaftsbau,Bildung,CH 1422 Grandson,VD,Kleinunternehmen,NaN,NaN,43965,43990.0,LE REPUIS,Un maître socioprofessionnel (H/F) à 100% sect...,Lehrer,<p><strong>Un maître socioprofessionnel (H/F) ...,Un maître socioprofessionnel HF à 100 secteur ...,A Socioprofessional Master HF to 100 Sector Ho...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,120,20933760,24310;24210,Kommunikationsfähigkeit;Planung;Datacap;Proble...,Bildung,Genève,GE,Mittelunternehmen,Unterrichtsassistent;Lagerist;Logistik-Jobs,BACHELOR;MASTER;BASICEDUCATION;MATURA;CERTIFIC...,44051,NaN,ESDA l’Ecole Supérieure de Droit et d’Administ...,Looking for a Market Data Analyst / Program Ma...,Programmverantwortlicher;Business Intelligence...,<p>Emploi</p> <p>ESDA is looking for a Market ...,Emploi ESDA is looking for a Market Data Analy...,Emploi ESDA is looking for a Market Data Analy...,NaN
121,121,21930428,26220;41200,Git;Texten und Überarbeiten;Problemstrukturier...,NPO,CH-1211 Genève,GE,Mittelunternehmen,Informatiker-Jobs;Projektmanagement-Jobs;Daten...,BACHELOR;MASTER;COLLEGE;DIPLOMA;UNIVERSITY,44251,44296.0,UNOG United Nations Office at Geneva,ASSISTANT DATA ANALYST,Data Scientist;Administrativer Assistent,Org. Setting and Reporting <p> This position i...,Org. Setting and Reporting This position is l...,Org. Setting and Reporting This position is l...,NaN
122,122,22071257,26220,Kommunikationsfähigkeit;Mobilität;Kommunikatio...,Grosshandel;Maschinenbau;Fahrzeugbau;Baugewerbe,CH-1208 Genève,GE,Grossunternehmen,NaN,BACHELOR;MASTER;BASICEDUCATION;MATURA;CERTIFIC...,44277,44298.0,Caterpillar Sarl,Caterpillar Graduate Program - Junior Business...,Absolvent;Data Scientist,<h1>Caterpillar Graduate Program - Junior Busi...,Caterpillar Graduate Program Junior Business ...,Caterpillar Graduate Program Junior Business ...,NaN
123,123,18359929,21490,Effektivität;Einrichtungen;Modellieren;Begeist...,Architektur und Planung,CH-1228 Plan-les-Ouates,GE,Kleinstunternehmen,Verlags-Jo

In [14]:
VikaDfLabeled = VikaDf[~VikaDf["MJ"].isna()]

In [15]:
VikaDfLabeled = VikaDfLabeled.rename(columns ={"ID":"JOB_ID"}) 


In [16]:
GreenDf = GreenDf.rename(columns ={"ID":"JOB_ID"}) 


In [17]:
GreenDf = GreenDf[["JOB_ID", "MJ", "ISCO", "TRANSLATED_JOBS", "CANTON", "COMPANY_SIZE" ]]

In [18]:
GreenDf["MJ"] = 1

In [19]:
GreenDf["JOB_CONTENT"] = GreenDf.apply(lambda x: x["TRANSLATED_JOBS"].split("."), axis = 1)

In [20]:
GreenDf.loc[0]["JOB_CONTENT"]

['The Federal Meteorology and Climatology Office Meteosuisse is looking for on October 1,2020 or to agree with a collaborator scientifier 80 Geneva WeeGewsuisse good to know as collaboratorComponent scientist in the postprocessing team and verification you are responsible for the development and the Software operation for automatic processing of the large amount of information from time digital prediction templates',
 ' The postprocessing and audit team is part of the Meteosuisse Forecast Development Division is responsible for exploiting new methods of automatic forecasts as well as decision support systems',
 ' Your tasks develop and implement methods or programs for the production of weather forecasts, for example by aggregating different sources of information for feeding the Web sitesuisse and its mobile application',
 ' Apply Statistical Methods of Machine Learning to improve weather forecasts but also to develop tools for analyzing and visualizing meteorological data for interna

In [21]:
VikaDfLabeled = VikaDfLabeled[["JOB_ID", "MJ"]]


In [22]:
EmilyDf = EmilyDf[["JOB_ID", "MJ"]]


In [23]:
#EmilyVikaDf = pd.merge(VikaDfLabeled, EmilyDf, on = "JOB_ID")

In [24]:
EmilyVikaDf = VikaDfLabeled.append(EmilyDf)

In [25]:
EmilyVikaDf

,JOB_ID,MJ
0,12047533,1
1,14422389,1
2,14616465,1
3,14617343,1
4,14515774,1
...,...,...
25,13762804,False
26,14239400,True
27,13823543,True
28,12723027,False


In [26]:
for label in ["pas de texte", "?", "False", False, "false", "FALSE", "non", "no", "no job offer text"]:
    EmilyVikaDf["MJ"] = EmilyVikaDf["MJ"].replace(label, 0)
for label in ["true", "True", True, "TRUE", "1"]:
    EmilyVikaDf["MJ"] = EmilyVikaDf["MJ"].replace(label, 1)

In [27]:
EmilyVikaDf = EmilyVikaDf[EmilyVikaDf["MJ"] != "same as above"]

In [28]:
EmilyVikaDf["MJ"].value_counts()

0    237
1    179
Name: MJ, dtype: int64

In [30]:
emilyVikaJobsMergedDf = EmilyVikaDf[["JOB_ID", "MJ"]].merge(allJobsDf, on="JOB_ID", how="outer")

In [ ]:
emilyVikaJobsMergedDf

In [31]:
mergedDf = emilyVikaJobsMergedDf[emilyVikaJobsMergedDf.columns.intersection(GreenDf.columns)].append(GreenDf)

In [32]:
mergedDf = mergedDf.reset_index()

In [33]:
mergedDf["NumSentences"] = mergedDf.apply(lambda x: len(x["JOB_CONTENT"]), axis=1)
mergedDf["NumCharacters"] = mergedDf.apply(lambda x: sum([len(y) for y in x["JOB_CONTENT"]]), axis=1)

In [34]:
mergedDf["MJ"].value_counts()

1    304
0    237
Name: MJ, dtype: int64

In [35]:
mergedDf["CANTON"] = mergedDf.apply(lambda x: x["CANTON"].split(";") if type(x["CANTON"]) == str else x["CANTON"], axis=1)

In [36]:
mergedDf["COMPANY_SIZE"] = mergedDf["COMPANY_SIZE"].replace("Grossunternehmen", "Large")
mergedDf["COMPANY_SIZE"] = mergedDf["COMPANY_SIZE"].replace("Kleinstunternehmen", "Micro")
mergedDf["COMPANY_SIZE"] = mergedDf["COMPANY_SIZE"].replace("Kleinunternehmen", "Small")
mergedDf["COMPANY_SIZE"] = mergedDf["COMPANY_SIZE"].replace("Mittelunternehmen", "Midsize")

In [37]:
mergedDf["CANTON1"] = mergedDf.apply(lambda x: x["CANTON"][0] if type(x["CANTON"]) != str else x["CANTON"], axis=1)

In [38]:
labeledDf = mergedDf[~mergedDf["MJ"].isna()]

In [39]:
labeledDf["MJ"] = labeledDf["MJ"].astype('bool')

<ipython-input-39-c9fa01c0230e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeledDf["MJ"] = labeledDf["MJ"].astype('bool')


In [40]:
labeledDf["MJ"].value_counts()

True     304
False    237
Name: MJ, dtype: int64

In [41]:
mergedDf[mergedDf["TRANSLATED_JOBS"] != ""]

,index,JOB_ID,MJ,ISCO,JOB_CONTENT,TRANSLATED_JOBS,CANTON,COMPANY_SIZE,NumSentences,NumCharacters,CANTON1
0,0,12047533.0,1,31230;12190;21620,[Building leaders Horticultural technician HF ...,Building leaders Horticultural technician HF T...,[BE],Micro,25,1827,BE
1,1,14422389.0,1,21620,[As an educational and social education center...,"As an educational and social education center,...",[BE],Small,15,2278,BE
2,2,14616465.0,1,21420,[Civil Engineer 1804 Corsiersurvey Engineering...,Civil Engineer 1804 Corsiersurvey Engineering ...,[VD],Micro,12,1347,VD
3,3,14617343.0,1,21620,"[24, 05, 2017 Internship Landscape Developme...",24. 05. 2017 Internship Landscape Development ...,[BE],Micro,7,500,BE
4,4,14515774.0,1,21430,[Company portrait The office for the forest of...,Company portrait The office for the forest of ...,[BE],Midsize,21,1848,BE
...,...,...,...,...,...,...,...,...,...,...,...
8685,120,20933760.0,1,24310;24210,[Emploi ESDA is looking for a Market Data Anal...,Emploi ESDA is looking for a Market Data Analy...,[GE],Midsize,12,3486,GE
8686,121,21930428.0,1,26220;41200,"[Org, Setting and Reporting This position is...",Org. Setting and Reporting This position is l...,[GE],Midsize,66,9579,GE
8687,122,22071257.0,1,26220,[Caterpillar Graduate Program Junior Business...,Caterpillar Graduate Program Junior Business ...,[GE],Large,21,4254,GE
8688,123,18359929.0,1,21490,"[Energy Energy Management Raymond E, Moser ha...",Energy Energy Management Raymond E. Moser has ...,[GE],Micro,24,2311,GE


In [43]:
mergedDf[mergedDf["TRANSLATED_JOBS"] != ""].to_csv("Data/mergedJobsAll.csv")

In [42]:
labeledDf.to_csv("Data/mergedAnnotatedJobsDf.csv")

In [ ]:
allvalues = mergedDf["CANTON1"].value_counts()
valuesGreen = mergedDf[mergedDf["MJ"] == True]["CANTON1"].value_counts()
valuesNotGreen = mergedDf[mergedDf["MJ"] == False]["CANTON1"].value_counts()
print(len(allvalues), len(valuesGreen), len(valuesNotGreen))
plt.bar(allvalues.index, allvalues, color='grey', label="Unlabeled jobs")
plt.bar(valuesGreen.index, valuesGreen, color='green', label="Environmental jobs")
plt.bar(valuesNotGreen.index, valuesNotGreen, bottom=valuesGreen[valuesNotGreen.index], color="brown", label="Non-environmental jobs")
plt.xlabel("Canton of job")
plt.ylabel("Number of jobs")
plt.title("Canton distribution of 1000 jobs")
plt.legend()
plt.savefig("Plots/CantonDist.pdf")

In [ ]:
valuesGreen

In [ ]:
labeledDf

In [ ]:
values = mergedDf["COMPANY_SIZE"].value_counts()
valuesGreen = mergedDf[mergedDf["MJ"] == True]["COMPANY_SIZE"].value_counts()
valuesNotGreen = mergedDf[mergedDf["MJ"] == False]["COMPANY_SIZE"].value_counts()
print(len(values), len(valuesGreen), len(valuesNotGreen))
plt.bar(values.index, values, color='grey', label="Unlabeled jobs")
plt.bar(valuesGreen.index, valuesGreen, color='green', label="Environnmental jobs")
plt.bar(valuesNotGreen.index, valuesNotGreen, bottom=valuesGreen[valuesNotGreen.index], color="brown", label="Non-environmental jobs")
plt.xlabel("Size of company")
plt.ylabel("Number of jobs")
plt.title("Distribution of size of company for 1000 jobs")
plt.savefig("Plots/CompanySize.pdf")
plt.legend()

In [ ]:
bins = list(range(0, 20, 2))+list(range(20, 50, 5))+list(range(50,100,10))
#plt.hist(mergedDf["NumSentences"], bins=bins, density=True, color="grey")
plt.hist(mergedDf[mergedDf["MJ"] == True]["NumSentences"], bins=bins, density=True, color="green", alpha=0.5, label="Environmental jobs")
plt.hist(mergedDf[mergedDf["MJ"] == False]["NumSentences"], bins=bins, density=True, color="brown", alpha=0.5, label="Non-environmental jobs")
plt.xlabel("Number of sentences in job description")
plt.ylabel("Normalized counts")
plt.legend()
plt.title("Distribution of number of sentences in job description")
plt.savefig("Plots/SentenceLength.pdf")


In [ ]:
bins = list(range(0, 40, 5))
#plt.hist(mergedDf["NumSentences"], bins=bins, density=True, color="grey")
plt.hist(mergedDf[mergedDf["MJ"] == True]["NumSentences"], bins=bins, density=True, color="green", alpha=0.5, label="Environmental jobs")
plt.hist(mergedDf[mergedDf["MJ"] == False]["NumSentences"], bins=bins, density=True, color="brown", alpha=0.5, label="Non-environmental jobs")
plt.xlabel("Number of sentences in job description")
plt.ylabel("Normalized counts")
plt.legend()
plt.title("Distribution of number of sentences in job description")
plt.savefig("Plots/SentenceLengthZoomed.pdf")


In [ ]:
#bins = list(range(0, 20, 2))+list(range(20, 50, 5))+list(range(50,100,10))
#plt.hist(mergedDf["NumSentences"], bins=bins, density=True, color="grey")
plt.hist(mergedDf[mergedDf["MJ"] == True]["NumCharacters"],  density=True, color="green", alpha=0.5, label="Environmental jobs")
plt.hist(mergedDf[mergedDf["MJ"] == False]["NumCharacters"], density=True, color="brown", alpha=0.5, label="Non-environmental jobs")
plt.xlabel("Number of characters in job description")
plt.ylabel("Normalized counts")
plt.legend()
plt.title("Distribution of number of characters in job description")
plt.savefig("Plots/CharLength.pdf")


In [ ]:
bins = list(range(0, 6000, 200))
plt.hist(mergedDf["NumCharacters"], bins=bins, density=True, color="grey", alpha = 0.5, label = "Unlabeled jobs")
plt.hist(mergedDf[mergedDf["MJ"] == True]["NumCharacters"], bins=bins, density=True, color="green", alpha=0.5, label="Environmental jobs")
plt.hist(mergedDf[mergedDf["MJ"] == False]["NumCharacters"], bins =bins, density=True, color="brown", alpha=0.5, label="Non-environmental jobs")
plt.xlabel("Number of characters in job description")
plt.ylabel("Normalized counts")
plt.legend()
plt.title("Distribution of number of characters in job description")
plt.savefig("Plots/CharLengthZoomed.pdf")


In [ ]:
allvalues = allJobsDf["CANTON1"].value_counts()
plt.bar(allvalues.index, allvalues, color='grey', label="Unlabeled jobs")
plt.xlabel("Canton of job")
plt.ylabel("Number of jobs")
plt.title("Canton distribution of all 8565 jobs")
plt.legend()
plt.savefig("Plots/CantonDistAll.pdf")


In [ ]:
allvalues = allJobsDf["COMPANY_SIZE"].value_counts()
plt.bar(allvalues.index, allvalues, color='grey', label="Unlabeled jobs")
plt.xlabel("Size of company")
plt.ylabel("Number of jobs")
plt.title("Company size distribution of all 8565 jobs")
plt.legend()
plt.savefig("Plots/CompanySizeAll.pdf")

In [ ]:
bins = list(range(0, 6000, 200))
#plt.hist(mergedDf["NumSentences"], bins=bins, density=True, color="grey")
plt.hist(allJobsDf["NumCharacters"], bins=bins, density=True, color="grey", label="Unlabeled jobs")
plt.xlabel("Number of characters in job description")
plt.ylabel("Normalized counts")
plt.legend()
plt.title("Distribution of number of characters in job description for all 8565 jobs")
plt.savefig("Plots/CharLengthAll.pdf")


In [ ]:
bins = list(range(0, 20, 2))+list(range(20, 50, 5))+list(range(50,100,10))
#plt.hist(mergedDf["NumSentences"], bins=bins, density=True, color="grey")
plt.hist(allJobsDf["NumSentences"], bins=bins, density=True, color="grey", label="Unlabeled jobs")
plt.xlabel("Number of sentences in job description")
plt.ylabel("Normalized counts")
plt.legend()
plt.title("Distribution of number of sentences in job description for all 8565 jobs")
plt.savefig("Plots/SentenceLengthAll.pdf")